In [1]:
import sys

sys.path.append("../../")

from app.common.logger import logger
from app.config.utils import Configuration, init_config
from app.common.llm_clients.openai_client import OpenAILLMClient

from ml.data_processing.prompt import STEP_1_2_SYNTHETIC_TEXTBOOK_PROMPT

In [2]:
config = Configuration()
init_config(config)
config = config()

# Data Load & Grouping

In [3]:
import json

with open("../data/amr_guide_preprocessed.json", "r", encoding="utf-8") as f:
    arm_guide_preprocessed = json.load(f)

with open("../data/acs_alarm_preprocessed.json", "r", encoding="utf-8") as f:
    acs_alarm_preprocessed = json.load(f)

In [13]:
import re

def group_by_h1(data_list):
    grouped_data = {}

    for item in data_list:
        original_text = item.get("original", "")

        # 정규표현식으로 첫 줄의 대제목(# ...) 추출
        # 예: "# 1. Daily Operating\n## 1.1 ..." -> "# 1. Daily Operating"
        match = re.search(r'^# .+', original_text)

        if match:
            h1_title = match.group(0).strip()
        else:
            # 대제목이 없는 경우 (예외 처리)
            h1_title = "Others"

        if h1_title not in grouped_data:
            grouped_data[h1_title] = []

        grouped_data[h1_title].append(item)

    return grouped_data

In [6]:
grouped_amr_guide = group_by_h1(arm_guide_preprocessed)

In [7]:
grouped_amr_guide.keys()

dict_keys(['# 1. Daily Operating', '# 2. Trouble Shooting: Call Assignment', '# 3. Trouble Shooting: Navigation & Obstacle', '# 4. Trouble Shooting: Docking & H/W', '# 5. Trouble Shooting: Model Specific', '# 6. Operational Manual: Control Tools (조작 및 유지보수 매뉴얼)', '# 7. Appendix'])

In [14]:
def group_alarms_by_range(data_list):
    grouped_data = {}

    # 알람 그룹 정의
    # 각 10000 단위로 그룹핑 (예: 10000~19999, 20000~29999 ...)

    for item in data_list:
        original_text = item.get("original", "")

        # 마크다운 테이블의 "No." 컬럼에서 숫자 추출
        # 정규식: | ... | 숫자 | ... | 형태에서 숫자만 찾음
        # 예: "| 테스크 | 10000 | ..." -> 10000 추출
        # 테이블의 각 행을 순회하며 알람 코드를 찾음
        alarm_codes = re.findall(r'\|\s*(\d{1,5})\s*\|', original_text)

        if not alarm_codes:
            # 알람 코드가 없는 경우 (예외 처리)
            group_key = "Others"
        else:
            # 해당 청크에 포함된 첫 번째 알람 코드를 기준으로 그룹 결정
            # (한 청크 내에는 보통 유사한 대역의 코드가 묶여 있음)
            first_code = int(alarm_codes[0])

            # 10000 단위로 그룹 키 생성 (10000, 20000, 30000 ...)
            # 예: 10015 -> 10000, 30040 -> 30000
            start_range = (first_code // 10000) * 10000
            group_key = str(start_range)

            # 예외: 10000 미만의 코드들 (ACS 감지 등)은 '0' 또는 'Others'로 처리하거나
            # 명시적으로 구분 가능. 여기서는 0~9999 대역으로 묶임.

        if group_key not in grouped_data:
            grouped_data[group_key] = []

        grouped_data[group_key].append(item)

    return grouped_data

In [15]:
grouped_acs_alarm = group_alarms_by_range(acs_alarm_preprocessed)

In [16]:
grouped_acs_alarm.keys()

dict_keys(['10000', '20000', '30000', '40000', '70000', '80000', '0'])

# 합성 데이터 생성

In [4]:
model = "google/gemini-3-pro-preview"
provider = "openrouter"
api_key = config["openrouter"]["api_key"]
params = {
    "max_tokens": 32768,
    "temperature": 0.0
}
llm_client = OpenAILLMClient(model=model, provider=provider, api_key=api_key, params=params)

2026-01-05 14:49:35.319 | INFO     | app.common.llm_clients.openai_client:_create_clients:57 - Creating OpenRouter client.


## 그룹 별 텍스트북 생성

In [ ]:
resuls = []
for k, v in grouped_amr_guide.items():
    context = [x['preprocessed'] for x in v]
    context = f"{k}\n" + "\n".join(context)
    user_prompt = f"""[입력 텍스트]
    {context}"""
    response = llm_client.generate(system_prompt=STEP_1_2_SYNTHETIC_TEXTBOOK_PROMPT, chat_messages=[{"role": "user", "content": user_prompt}])
    result = response.choices[0].message.content
    logger.info(result)

    json_result = {
        'id': k,
        'preprocessed_group': context,
        'textbook': result
    }
    resuls.append(json_result)

2026-01-02 17:42:55.515 | INFO     | __main__:<module>:9 - # BMA 자동화 시스템 기술 백서: 운영 로직 및 전력 관리 아키텍처
## (Technical Whitepaper: Operational Logic and Power Management Architecture in BMA Systems)

**작성자:** 수석 아키텍트 (Chief Architect)
**문서 등급:** Confidential (Internal Use Only)
**적용 대상:** 현대자동차그룹 BMA 공정 0.3T/1T AMR 및 WEB ACS 운영 엔지니어

---

### 1. 서론 (Introduction)

본 챕터에서는 BMA(Battery Module Assembly) 공정의 핵심 물류 자원인 AMR(Autonomous Mobile Robot)과 이를 관제하는 상위 시스템 WEB ACS(Activity Control System) 간의 **논리적 상호작용(Logical Interaction)** 및 **전력 관리 메커니즘(Power Management Mechanism)**을 다룬다.

단순한 조작 절차를 넘어, 시스템이 미션(Mission)을 생성하고 할당하는 알고리즘적 배경과, 리튬이온 배터리 기반의 모빌리티 하드웨어가 전기적 안정성을 유지하기 위해 요구되는 물리적 제약 사항을 공학적 관점에서 심층 분석한다.

---

### 2. 미션 디스패칭 아키텍처와 Place Cycle 제어 논리
#### (Mission Dispatching Architecture & Place Cycle Control Logic)

WEB ACS는 공정 내 수백 개의 노드(Node)와 링크(Link)를 관리하며, 각 위치(Place)의 상태에 따라 AMR에게 작업을 할당한다. 이때 **'Place Cycle'** 설정은 시스템이 물리적 공간을 논리적 가용 자원으로 인식하게 만드는 핵심 인터페이스다.

#### 2.1. 이중 검증 게이트웨이: Lo

In [ ]:
import json

with open("amr_guide_textbook.json", "w", encoding="utf-8") as f:
    json.dump(resuls, f, ensure_ascii=False, indent=4)

In [5]:
with open("../data/amr_guide_textbook.json", "r", encoding="utf-8") as f:
    resuls = json.load(f)

In [ ]:
results2 = []
for k, v in grouped_acs_alarm.items():
    context = [x['preprocessed'] for x in v]
    context = f"{k}\n" + "\n".join(context)
    user_prompt = f"""[입력 텍스트]
    {context}"""
    logger.info(user_prompt)
    logger.info('====')
    response = llm_client.generate(system_prompt=STEP_1_2_SYNTHETIC_TEXTBOOK_PROMPT, chat_messages=[{"role": "user", "content": user_prompt}])
    result = response.choices[0].message.content
    logger.info(result)
    json_result = {
        'id': k,
        'preprocessed_group': context,
        'textbook': result
    }
    results2.append(json_result)

2026-01-02 17:54:46.416 | INFO     | __main__:<module>:7 - [입력 텍스트]
    10000
BMA 공정 AMR의 [10000] RFID 감지 안됨(RFID not detected) 경고는 주행 경로상의 RFID 태그 인식이 불가능한 상태를 의미한다. **시스템 자동 복구가 실패하여 수동 조작이 필요한 경우,** 운영자는 AMR의 정지 위치를 확인하고 RFID 리더기와 태그의 정렬 상태를 점검해야 한다.

BMA 공정 AMR의 [10001] 리프트 미션 실패(Lift mission fail) 경고는 팔레트 상하강 동작이 지정된 시간 내에 완료되지 않았음을 나타낸다. **시스템 자동 복구가 실패하여 수동 조작이 필요한 경우,** 운영자는 리프트 구동부의 기계적 간섭 여부를 확인한 후 HMI를 통해 리프트 위치를 초기화해야 한다.

BMA 공정 AMR의 [10002] 턴테이블 미션 실패(Turntable mission fail) 경고는 상단 턴테이블의 회전 동작이 정상 범위 내에서 종료되지 않았을 때 발생한다. **시스템 자동 복구가 실패하여 수동 조작이 필요한 경우,** 운영자는 턴테이블의 회전 반경 내 이물질을 제거하고 수동 모드에서 원점 복귀를 수행해야 한다.

BMA 공정 AMR의 [10003] 도킹 중 장애물 감지(Obstacle detected during docking) 경고는 설비 진입 및 도킹 과정에서 안전 센서가 물체를 감지하여 주행이 중단된 상태를 의미한다. **시스템 자동 복구가 실패하여 수동 조작이 필요한 경우,** 운영자는 도킹 경로상의 장애물을 제거한 후 주행 재개 버튼을 조작해야 한다.

BMA 공정 AMR의 [10004] 거치대 도킹 실패(Wing docking fail) 경고는 AMR이 윙(Wing) 타입 거치대와의 물리적 결합 위치 정밀도 확보에 실패했음을 나타낸다. **시스템 자동 복구가 실패하여 수동 조작이 필요한 경우,** 운영자는 거치대의 정위치 이탈 여부를 확인하고 AMR을 

In [22]:
import json

with open("acs_alarm_textbook.json", "w", encoding="utf-8") as f:
    json.dump(results2, f, ensure_ascii=False, indent=4)

In [6]:
with open("../data/acs_alarm_textbook.json", "r", encoding="utf-8") as f:
    result2 = json.load(f)

In [12]:
result2

[{'id': '10000',
  'preprocessed_group': '10000\nBMA 공정 AMR의 [10000] RFID 감지 안됨(RFID not detected) 경고는 주행 경로상의 RFID 태그 인식이 불가능한 상태를 의미한다. **시스템 자동 복구가 실패하여 수동 조작이 필요한 경우,** 운영자는 AMR의 정지 위치를 확인하고 RFID 리더기와 태그의 정렬 상태를 점검해야 한다.\n\nBMA 공정 AMR의 [10001] 리프트 미션 실패(Lift mission fail) 경고는 팔레트 상하강 동작이 지정된 시간 내에 완료되지 않았음을 나타낸다. **시스템 자동 복구가 실패하여 수동 조작이 필요한 경우,** 운영자는 리프트 구동부의 기계적 간섭 여부를 확인한 후 HMI를 통해 리프트 위치를 초기화해야 한다.\n\nBMA 공정 AMR의 [10002] 턴테이블 미션 실패(Turntable mission fail) 경고는 상단 턴테이블의 회전 동작이 정상 범위 내에서 종료되지 않았을 때 발생한다. **시스템 자동 복구가 실패하여 수동 조작이 필요한 경우,** 운영자는 턴테이블의 회전 반경 내 이물질을 제거하고 수동 모드에서 원점 복귀를 수행해야 한다.\n\nBMA 공정 AMR의 [10003] 도킹 중 장애물 감지(Obstacle detected during docking) 경고는 설비 진입 및 도킹 과정에서 안전 센서가 물체를 감지하여 주행이 중단된 상태를 의미한다. **시스템 자동 복구가 실패하여 수동 조작이 필요한 경우,** 운영자는 도킹 경로상의 장애물을 제거한 후 주행 재개 버튼을 조작해야 한다.\n\nBMA 공정 AMR의 [10004] 거치대 도킹 실패(Wing docking fail) 경고는 AMR이 윙(Wing) 타입 거치대와의 물리적 결합 위치 정밀도 확보에 실패했음을 나타낸다. **시스템 자동 복구가 실패하여 수동 조작이 필요한 경우,** 운영자는 거치대의 정위치 이탈 여부를 확인하고 AMR을 후진시킨 후 재도킹을 시도해야 한다.\

## 단락 별 텍스트북 생성

In [8]:
len(resuls)

7

In [9]:
for i, x in enumerate(arm_guide_preprocessed):
    context = x['preprocessed']
    user_prompt = f"""챕터 번호는 없어도 돼.

[입력 텍스트]
{context}"""
    logger.info(user_prompt)
    response = llm_client.generate(system_prompt=STEP_1_2_SYNTHETIC_TEXTBOOK_PROMPT, chat_messages=[{"role": "user", "content": user_prompt}])
    result = response.choices[0].message.content
    logger.info(result)
    json_result = {
        'id': i,
        'preprocessed_group': context,
        'textbook': result
    }
    resuls.append(json_result)
    logger.info(f"{len(resuls)} 텍스트북 생성 완료")

2026-01-05 14:49:46.809 | INFO     | __main__:<module>:7 - 챕터 번호는 없어도 돼.

[입력 텍스트]
WEB ACS의 Place Cycle 설정 시, AMR이 미션을 수신하고 동작하기 위해서는 Logistics 스위치와 Run 상태가 모두 ON으로 활성화되어야 한다.

WEB ACS의 Place Cycle 설정 절차는 오른쪽 톱니바퀴 아이콘 클릭 후 Place Cycle 메뉴를 선택하고, 검색창에서 제어할 Place 이름을 검색하여 Logistics 스위치와 Run 상태를 각각 ON과 RUNNING으로 전환하는 순서로 진행한다.

Place Cycle의 Logistics 기능은 시스템이 미션을 생성하고 AMR이 이를 수신하여 수행하도록 활성화하는 장치이며, 생산 가동 중에는 반드시 ON, 비가동 또는 종업 시에는 OFF 상태를 유지해야 한다.

Place Cycle의 RUN 상태는 가동 중인 생산 라인에서 필수적으로 활성화해야 하는 항목이며, 대부분의 운영 상황에서 상시 ON 상태를 유지한다.

생산 시작 전에는 Place Cycle의 Logistics와 Run 버튼을 ON으로 활성화하고, 생산 완료 후에는 해당 버튼들을 OFF로 변경하는 것을 운영 규칙으로 한다.
2026-01-05 14:50:21.377 | INFO     | __main__:<module>:10 - # WEB ACS 기반의 Place Cycle 제어 논리와 미션 디스패칭(Mission Dispatching) 메커니즘

## 1. 서론: BMA 공정 내 노드(Node) 관리의 중요성

현대자동차그룹 BMA(Battery Module Assembly) 공정의 자동화 시스템에서 **WEB ACS(Automated Control System)**는 수십 대의 AMR(Autonomous Mobile Robot)과 설비 간의 물류 흐름을 조율하는 중앙 관제탑 역할을 수행한다. 이때, 공장 내의 물리적 위치(Station)를 논리적 객체로 추상화한 것이 

In [10]:
with open("amr_guide_textbook_merged.json", "w", encoding="utf-8") as f:
    json.dump(resuls, f, ensure_ascii=False, indent=4)

In [13]:
len(result2)

7

In [14]:
for i, x in enumerate(acs_alarm_preprocessed):
    context = x['preprocessed']
    user_prompt = f"""챕터 번호는 없어도 돼.

[입력 텍스트]
{context}"""
    logger.info(user_prompt)
    response = llm_client.generate(system_prompt=STEP_1_2_SYNTHETIC_TEXTBOOK_PROMPT, chat_messages=[{"role": "user", "content": user_prompt}])
    result = response.choices[0].message.content
    logger.info(result)

    json_result = {
        'id': i,
        'preprocessed_group': context,
        'textbook': result
    }
    result2.append(json_result)
    logger.info(f"{len(result2)} 텍스트북 생성 완료")

2026-01-05 15:26:36.122 | INFO     | __main__:<module>:7 - 챕터 번호는 없어도 돼.

[입력 텍스트]
BMA 공정 AMR의 [10000] RFID 감지 안됨(RFID not detected) 경고는 주행 경로상의 RFID 태그 인식이 불가능한 상태를 의미한다. **시스템 자동 복구가 실패하여 수동 조작이 필요한 경우,** 운영자는 AMR의 정지 위치를 확인하고 RFID 리더기와 태그의 정렬 상태를 점검해야 한다.

BMA 공정 AMR의 [10001] 리프트 미션 실패(Lift mission fail) 경고는 팔레트 상하강 동작이 지정된 시간 내에 완료되지 않았음을 나타낸다. **시스템 자동 복구가 실패하여 수동 조작이 필요한 경우,** 운영자는 리프트 구동부의 기계적 간섭 여부를 확인한 후 HMI를 통해 리프트 위치를 초기화해야 한다.

BMA 공정 AMR의 [10002] 턴테이블 미션 실패(Turntable mission fail) 경고는 상단 턴테이블의 회전 동작이 정상 범위 내에서 종료되지 않았을 때 발생한다. **시스템 자동 복구가 실패하여 수동 조작이 필요한 경우,** 운영자는 턴테이블의 회전 반경 내 이물질을 제거하고 수동 모드에서 원점 복귀를 수행해야 한다.

BMA 공정 AMR의 [10003] 도킹 중 장애물 감지(Obstacle detected during docking) 경고는 설비 진입 및 도킹 과정에서 안전 센서가 물체를 감지하여 주행이 중단된 상태를 의미한다. **시스템 자동 복구가 실패하여 수동 조작이 필요한 경우,** 운영자는 도킹 경로상의 장애물을 제거한 후 주행 재개 버튼을 조작해야 한다.

BMA 공정 AMR의 [10004] 거치대 도킹 실패(Wing docking fail) 경고는 AMR이 윙(Wing) 타입 거치대와의 물리적 결합 위치 정밀도 확보에 실패했음을 나타낸다. **시스템 자동 복구가 실패하여 수동 조작이 필요한 경우,** 운영자는 거치대의 정위치 이탈 여부를 확인하고 

In [15]:
with open("acs_alarm_textbook_merged.json", "w", encoding="utf-8") as f:
    json.dump(result2, f, ensure_ascii=False, indent=4)